# imports
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from random import random
from random import seed
from math import exp
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

In [3]:
# No. of rows(rows)
no_of_samples = 200

# No. of features {F1,F2,F3,F4}=4
no_of_features = 4

# No. of redundent features (Extra Feature)
no_of_extra = 1

#types of classes
types_of_classes = 2

In [4]:

X, y = make_classification(n_samples=no_of_samples, n_features=no_of_features,
                                            n_redundant=no_of_extra, n_classes=types_of_classes)
df = pd.DataFrame(X, columns=['F1', 'F2', 'F3', 'F4'])


In [5]:
#reading the csv file by using pandas 

df=pd.read_csv('assignment.csv',index_col=0)
df.head()

,F1,F2,F3,F4,L1
S No.,,,,,
0,0.989952,-0.216663,-0.840217,0.247416,0
1,-0.053900,-1.854247,1.481230,0.798278,1
2,-1.020268,1.747037,-2.276463,-0.618126,0
3,-0.514083,-1.461922,-1.395697,1.019334,0
4,-0.771490,0.828319,1.436928,-0.675839,1


In [6]:
#creating the network for weights

def create_W(n_inp, n_hid, n_out):
    net=list()
    H_layer = [{'W':[random() for i in range(n_inp + 1)]} for i in range(n_hid)]
    net.append(H_layer)
    O_layer = [{'W':[random() for i in range(n_hid + 1)]} for i in range(n_out)]
    net.append(O_layer)
    return net

In [7]:

#activation function for MLP

def activate(W, inp):
    activation=W[-1]
    for i in range(len(W)-1):
        activation+=W[i]*inp[i]
    return activation

def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))


In [8]:

#forward propogation to pass it on next layer using activation function

def propagation_next_layer(net,raw):
    inp=raw
    for layer in net:
        new_inp=[]
        for neuron in layer:
            activation=activate(neuron['W'], inp)
            neuron['output']=transfer(activation)
            new_inp.append(neuron['output'])
        inp=new_inp
    return inp

def transfer_derivative(out):
    return out * (1.0 - out)

In [9]:


#backward propogation to learn for fine tuning the weights

def backward_learn(net, expected):
    for i in reversed(range(len(net))):
        layer = net[i]
        errors = list()
        if i != len(net)-1:
            for ind in range(len(layer)):
                err = 0.0
                for neuron in net[i + 1]:
                    err += (neuron['W'][ind] * neuron['delta'])
                errors.append(err)
        else:
            for index in range(len(layer)):
                neuron = layer[index]
                errors.append(expected[index] - neuron['output'])
        for index in range(len(layer)):
            neuron = layer[index]
            neuron['delta'] = errors[index] * transfer_derivative(neuron['output'])

In [10]:

#update weights W on training the algo

def update_W_train(net, row, l_rate):
    for i in range(len(net)):
        inp=row[:-1]
        if i!=0:
            inp=[neuron['output'] for neuron in net[i-1]]
        for neuron in net[i]:
            for j in range(len(inp)):
                neuron['W'][j]+=l_rate*neuron['delta']*inp[j]
            neuron['W'][-1]+=l_rate*neuron['delta']


In [11]:
#training the network

def train_MLP(net, train, l_rate, n_epoch, n_out):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            out = propagation_next_layer(net, row)
            expected = [0 for i in range(n_out)]
            expected[int(row[-1])] = 1
            sum_error += sum([(expected[i]-out[i])**2 for i in range(len(expected))])
            backward_learn(net, expected)
            update_W_train(net, row, l_rate)
        print('=>epoch=%d, =>lrate=%.3f, =>error=%.3f' % (epoch, l_rate, sum_error))


In [12]:

#predicting function

def prediction_def(net, row):
    out = propagation_next_layer(net, row)
    return out.index(max(out))

In [13]:
dataset=np.array(df[:])
dataset

array([[ 9.89951561e-01, -2.16663363e-01, -8.40216728e-01,
         2.47416467e-01,  0.00000000e+00],
       [-5.38998510e-02, -1.85424690e+00,  1.48123049e+00,
         7.98277529e-01,  1.00000000e+00],
       [-1.02026781e+00,  1.74703732e+00, -2.27646339e+00,
        -6.18125619e-01,  0.00000000e+00],
       [-5.14082520e-01, -1.46192216e+00, -1.39569663e+00,
         1.01933443e+00,  0.00000000e+00],
       [-7.71490282e-01,  8.28319117e-01,  1.43692804e+00,
        -6.75838889e-01,  1.00000000e+00],
       [ 8.48094534e-01, -1.78161924e+00, -2.62461131e+00,
         1.38275642e+00,  0.00000000e+00],
       [-1.93022008e-01, -9.74753567e-01,  5.83147698e-01,
         4.49385607e-01,  1.00000000e+00],
       [-6.03853857e-01, -1.70014910e-01, -2.61601874e+00,
         4.91796189e-01,  0.00000000e+00],
       [ 7.16873650e-01, -8.74769329e-01, -1.06727973e+00,
         6.45250699e-01,  0.00000000e+00],
       [-9.99038769e-01, -1.11815481e+00, -1.08217200e+00,
         7.81872044e-01

In [14]:
n_inp = len(dataset[0]) - 1
n_out = len(set([row[-1] for row in dataset]))
print(n_inp,n_out)


4 2


In [15]:
#splitting into test and train datset
#the data is split in the ratio 3:1
train_dataset=dataset[:150]
test_dataset=dataset[150:]

In [16]:
#feeding the datset into the network
net=create_W(n_inp,1,n_out)
train_MLP(net, train_dataset, 0.5, 100, n_out)

=>epoch=0, =>lrate=0.500, =>error=77.718
=>epoch=1, =>lrate=0.500, =>error=53.646
=>epoch=2, =>lrate=0.500, =>error=33.279
=>epoch=3, =>lrate=0.500, =>error=26.737
=>epoch=4, =>lrate=0.500, =>error=24.241
=>epoch=5, =>lrate=0.500, =>error=23.044
=>epoch=6, =>lrate=0.500, =>error=22.368
=>epoch=7, =>lrate=0.500, =>error=21.920
=>epoch=8, =>lrate=0.500, =>error=21.575
=>epoch=9, =>lrate=0.500, =>error=21.316
=>epoch=10, =>lrate=0.500, =>error=21.140
=>epoch=11, =>lrate=0.500, =>error=21.022
=>epoch=12, =>lrate=0.500, =>error=20.927
=>epoch=13, =>lrate=0.500, =>error=20.815
=>epoch=14, =>lrate=0.500, =>error=20.670
=>epoch=15, =>lrate=0.500, =>error=20.500
=>epoch=16, =>lrate=0.500, =>error=20.320
=>epoch=17, =>lrate=0.500, =>error=20.144
=>epoch=18, =>lrate=0.500, =>error=19.982
=>epoch=19, =>lrate=0.500, =>error=19.840
=>epoch=20, =>lrate=0.500, =>error=19.720
=>epoch=21, =>lrate=0.500, =>error=19.618
=>epoch=22, =>lrate=0.500, =>error=19.534
=>epoch=23, =>lrate=0.500, =>error=19.464
=>

In [17]:
#learned weights of the network
for layer in net:
    print(layer)

[{'W': [0.26023774893564044, -1.416259372199305, 10.830561108799797, -0.17579757526175338, -2.406868292931135], 'output': 0.9999995694939275, 'delta': 5.981062014024875e-09}]
[{'W': [-5.992630992984874, 2.6644014905663544], 'output': 0.034654194050033775, 'delta': -0.0011592964873913263}, {'W': [5.992634961165923, -2.6644030957380123], 'output': 0.9653458851743792, 'delta': 0.0011592912819055072}]


In [18]:
#applying on training dataset ie. 150 rows

y_train=[]
pred=[]
for row in train_dataset:
    prediction = prediction_def(net, row)
    y_train.append(int(row[-1]))
    pred.append(prediction)


In [19]:
#Final Results on training-->

print("Accuracy Result: ",accuracy_score(y_train,pred))
print("Confusion Matrix: ",confusion_matrix(y_train,pred))
print("Precision Result: ",precision_score(y_train, pred))
print("recall Result: ",recall_score(y_train, pred))

Accuracy Result:  0.9266666666666666
Confusion Matrix:  [[68  5]
 [ 6 71]]
Precision Result:  0.9342105263157895
recall Result:  0.922077922077922


In [20]:
#applying on testing dataset ie. 50 rows
y_test=[]
pred=[]
for row in test_dataset:
    prediction = prediction_def(net, row)
    y_test.append(row[-1])
    pred.append(prediction)


In [21]:
#Final Results on testing-->


print("Accuracy Result: ",accuracy_score(y_test,pred))
print("Confusion Matrix: ",confusion_matrix(y_test,pred))
print("Precision Result: ",precision_score(y_test, pred))
print("recall Result: ",recall_score(y_test, pred))

Accuracy Result:  0.94
Confusion Matrix:  [[25  2]
 [ 1 22]]
Precision Result:  0.9166666666666666
recall Result:  0.9565217391304348
